In [1]:
!nvcc --version
!nvidia-smi
import torch
torch.__version__

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Thu May  2 09:37:08 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   33C    P8              11W /  7

'2.2.1+cu121'

In [2]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git@5aeb252b194b93dc2879b4ac34bc51a31b5aee13'

  Cloning https://github.com/facebookresearch/detectron2.git (to revision 5aeb252b194b93dc2879b4ac34bc51a31b5aee13) to /tmp/pip-req-build-0_4nmbnz
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-0_4nmbnz
  Running command git rev-parse -q --verify 'sha^5aeb252b194b93dc2879b4ac34bc51a31b5aee13'
  Running command git fetch -q https://github.com/facebookresearch/detectron2.git 5aeb252b194b93dc2879b4ac34bc51a31b5aee13
  Running command git checkout -q 5aeb252b194b93dc2879b4ac34bc51a31b5aee13
  Resolved https://github.com/facebookresearch/detectron2.git to commit 5aeb252b194b93dc2879b4ac34bc51a31b5aee13
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 13.4 MB/s eta 

In [3]:
%cd /content/
!git clone https://github.com/DomMcOyle/TACO-expl.git
%cd /content/TACO-expl
!git checkout maskdino
!git pull origin maskdino
%cd /content/TACO-expl/MaskDINO
!pip install -r requirements.txt
%cd /content/TACO-expl/MaskDINO/maskdino/modeling/pixel_decoder/ops
!sh make.sh

/content
Cloning into 'TACO-expl'...
remote: Enumerating objects: 1963, done.
remote: Counting objects: 100% (1388/1388), done.
remote: Compressing objects: 100% (778/778), done.
remote: Total 1963 (delta 690), reused 1282 (delta 594), pack-reused 575
Receiving objects: 100% (1963/1963), 122.46 MiB | 23.29 MiB/s, done.
Resolving deltas: 100% (1063/1063), done.
/content/TACO-expl
Branch 'maskdino' set up to track remote branch 'maskdino' from 'origin'.
Switched to a new branch 'maskdino'
From https://github.com/DomMcOyle/TACO-expl
 * branch            maskdino   -> FETCH_HEAD
Already up to date.
/content/TACO-expl/MaskDINO
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 kB 2.7 MB/s eta 0:00:00
/content/TACO-expl/MaskDINO/maskdino/modeling/pixel_decoder/ops
running build
running build_py
creating build
creating build/lib.linux-x86_64-cpython-310
creating build/lib.linux-x86_64-cpython-310/modules
copying modules/__init__.py -> build/lib.linux-x86_64-cpython-310/modules
copying mo

In [1]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
torch:  2.2 ; cuda:  cu121
detectron2: 0.6


In [2]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [3]:
import sys
sys.path.append("/content/TACO-expl/MaskDINO")
!cd /content/TACO-expl/MaskDINO/
cfg_biggest_path = '/content/TACO-expl/MaskDINO/configs/coco/instance-segmentation/swin/maskdino_R50_bs16_50ep_4s_dowsample1_2048.yaml'
model_biggest_weights_url = 'https://github.com/IDEA-Research/detrex-storage/releases/download/maskdino-v0.1.0/maskdino_swinl_50ep_300q_hid2048_3sd1_instance_maskenhanced_mask52.3ap_box59.0ap.pth'

cfg_smallest_path = '/content/TACO-expl/MaskDINO/configs/coco/instance-segmentation/maskdino_R50_bs16_50ep_3s.yaml'
model_smallest_weight_url = 'https://github.com/IDEA-Research/detrex-storage/releases/download/maskdino-v0.1.0/maskdino_r50_50ep_300q_hid1024_3sd1_instance_maskenhanced_mask46.1ap_box51.5ap.pth'

In [4]:
sys.path.append('/content/TACO-expl/MaskDINO/maskdino/')
from config import add_maskdino_config
%cd /content/TACO-expl/MaskDINO/maskdino/
from detectron2.projects.deeplab import add_deeplab_config
from maskdino import MaskDINO

def setup_cfg(cfg_path, weights_url):
    # load config from file and command-line arguments
    cfg = get_cfg()
    add_deeplab_config(cfg)
    add_maskdino_config(cfg)
    cfg.merge_from_file(cfg_path)
    #cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
    #cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.5
    cfg.MODEL.WEIGHTS = weights_url
    cfg.freeze()
    return cfg

cfg_alt_dino = setup_cfg(cfg_smallest_path, model_smallest_weight_url)
cfg_modified = cfg_alt_dino.copy()
print(cfg_modified["MODEL"])

/content/TACO-expl/MaskDINO/maskdino


/content/TACO-expl/MaskDINO/maskdino/modeling/criterion.py:346: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if self.dn is not "no" and mask_dict is not None:


ANCHOR_GENERATOR:
  ANGLES: [[-90, 0, 90]]
  ASPECT_RATIOS: [[0.5, 1.0, 2.0]]
  NAME: DefaultAnchorGenerator
  OFFSET: 0.0
  SIZES: [[32, 64, 128, 256, 512]]
BACKBONE:
  FREEZE_AT: 0
  NAME: build_resnet_backbone
DEVICE: cuda
FPN:
  FUSE_TYPE: sum
  IN_FEATURES: []
  NORM: 
  OUT_CHANNELS: 256
KEYPOINT_ON: False
LOAD_PROPOSALS: False
MASK_ON: False
META_ARCHITECTURE: MaskDINO
MaskDINO:
  BOX_LOSS: True
  BOX_WEIGHT: 5.0
  CLASS_WEIGHT: 4.0
  COST_BOX_WEIGHT: 5.0
  COST_CLASS_WEIGHT: 4.0
  COST_DICE_WEIGHT: 5.0
  COST_GIOU_WEIGHT: 2.0
  COST_MASK_WEIGHT: 5.0
  DEC_LAYERS: 9
  DEEP_SUPERVISION: True
  DICE_WEIGHT: 5.0
  DIM_FEEDFORWARD: 2048
  DN: seg
  DN_NOISE_SCALE: 0.4
  DN_NUM: 100
  DROPOUT: 0.0
  ENC_LAYERS: 0
  ENFORCE_INPUT_PROJ: False
  EVAL_FLAG: 1
  GIOU_WEIGHT: 2.0
  HIDDEN_DIM: 256
  IMPORTANCE_SAMPLE_RATIO: 0.75
  INITIALIZE_BOX_TYPE: bitmask
  INITIAL_PRED: True
  LEARN_TGT: False
  MASK_WEIGHT: 5.0
  NHEADS: 8
  NO_OBJECT_WEIGHT: 0.1
  NUM_OBJECT_QUERIES: 300
  OVERSAMPL

In [5]:
from google.colab import drive
drive.mount("/content/MyDrive/", force_remount = True)

Mounted at /content/MyDrive/


In [6]:
%cd /content/TACO-expl
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image

import os.path
import json
import argparse
import numpy as np
import random
import datetime as dt
import copy
import torch.nn as nn
from sklearn.model_selection import train_test_split
from pathlib import Path

import torch
import math
import torch.utils.data
from torch.utils.data import DataLoader
from pycocotools import mask as coco_mask


from torchvision.ops import RoIAlign
from HDDETR.datasets.torchvision_datasets.coco import CocoDetection as TvCocoDetection # from torchvision.datasets import CocoDetection as TvCocoDetection
                                                                                       # the right above import import does not work bc of attribute definitions
import HDDETR.datasets.transforms as T #from torchvision import transforms as T # this import does not contain the randomselect method
import HDDETR.util.misc as mutils

/content/TACO-expl


In [7]:
DEFAULT_RANDOM_SEED = 42
# basic random seed
def seedBasic(seed=DEFAULT_RANDOM_SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
# torch random seed
def seedTorch(seed=DEFAULT_RANDOM_SEED):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
# combine
def seedEverything(seed=DEFAULT_RANDOM_SEED):
    seedBasic(seed)
    seedTorch(seed)

seedEverything()

In [8]:
keep_categories = ["Bottle", "Bottle cap", "Can", "Cigarette", "Cup",
                   "Lid", "Plastic bag & wrapper", "Pop tab", "Straw"]

def create_map(original, keep_supercategories):
  class_map = {}
  for cat in original:
    if cat["supercategory"] in keep_supercategories:
      class_map[cat["name"]] = cat["supercategory"]
    else:
      class_map[cat["name"]] = "Other"
  return class_map

def replace_dataset_classes(dataset, class_map):
      """ Replaces classes of dataset based on a dictionary"""
      class_new_names = list(set(class_map.values()))
      class_new_names.sort()
      class_originals = copy.deepcopy(dataset['categories'])
      dataset['categories'] = []
      class_ids_map = {}  # map from old id to new id

      # Assign background id 0
      has_background = False
      if 'Background' in class_new_names:
          if class_new_names.index('Background') != 0:
              class_new_names.remove('Background')
              class_new_names.insert(0, 'Background')
          has_background = True

      # Replace categories
      for id_new, class_new_name in enumerate(class_new_names):
          # Make sure id:0 is reserved for background
          id_rectified = id_new
          if not has_background:
              id_rectified += 1

          category = {
              'supercategory': '',
              'id': id_rectified,  # Background has id=0
              'name': class_new_name,
          }
          dataset['categories'].append(category)
          # Map class names
          for class_original in class_originals:
              if class_map[class_original['name']] == class_new_name:
                  class_ids_map[class_original['id']] = id_rectified

      # Update annotations category id tag
      for ann in dataset['annotations']:
          ann['category_id'] = class_ids_map[ann['category_id']]

In [9]:
with open("/content/TACO-expl/data/annotations.json", "r") as f: #"/content/TACO/data/annotations.json"
    dataset = json.loads(f.read())

class_map = create_map(dataset["categories"], keep_categories)
replace_dataset_classes(dataset, class_map)
dataset["categories"]

[{'supercategory': '', 'id': 1, 'name': 'Bottle'},
 {'supercategory': '', 'id': 2, 'name': 'Bottle cap'},
 {'supercategory': '', 'id': 3, 'name': 'Can'},
 {'supercategory': '', 'id': 4, 'name': 'Cigarette'},
 {'supercategory': '', 'id': 5, 'name': 'Cup'},
 {'supercategory': '', 'id': 6, 'name': 'Lid'},
 {'supercategory': '', 'id': 7, 'name': 'Other'},
 {'supercategory': '', 'id': 8, 'name': 'Plastic bag & wrapper'},
 {'supercategory': '', 'id': 9, 'name': 'Pop tab'},
 {'supercategory': '', 'id': 10, 'name': 'Straw'}]

In [10]:
classes = [elem["name"] for elem in dataset["categories"]]
print(classes)

['Bottle', 'Bottle cap', 'Can', 'Cigarette', 'Cup', 'Lid', 'Other', 'Plastic bag & wrapper', 'Pop tab', 'Straw']


In [11]:
class TACODataset(TvCocoDetection):
    def __init__(
        self,
        img_folder,
        ann_file,
        classes,
        transforms,
        cache_mode=False,
        local_rank=0,
        local_size=1,
        use_crowd=False,
        return_unmodified=True # Needed for debugging purposes
    ):
        super(TACODataset, self).__init__(
            img_folder,
            ann_file,
            cache_mode=cache_mode,
            local_rank=local_rank,
            local_size=local_size,
        )
        self._transforms = transforms
        self.prepare = ConvertCocoPolysToMask(use_crowd)
        self.return_unmodified = return_unmodified
        self.image_dir = img_folder
        self.annotation_file = ann_file
        self.classes = classes

    def __getitem__(self, idx):
        img, target = super(TACODataset, self).__getitem__(idx)
        unmodified_img = np.array(img.copy())
        image_id = self.ids[idx]
        target = {"image_id": image_id, "annotations": target}
        img, target = self.prepare(img, target)
        if self._transforms is not None:
            img, target = self._transforms(img, target)
        if self.return_unmodified:
          return unmodified_img, img, target
        return img, target



class ConvertCocoPolysToMask(object):
    def __init__(self, use_crowd):
       self.use_crowd = use_crowd

    def __call__(self, image, target):
        w, h = image.size

        image_id = target["image_id"]
        image_id = torch.tensor([image_id])

        anno = target["annotations"]

        if not self.use_crowd:
          anno = [obj for obj in anno if "iscrowd" not in obj or obj["iscrowd"] == 0]

        # condition as sanity check for empty segmentations
        boxes = [obj["bbox"] for obj in anno if obj["segmentation"]]
        # guard against no boxes via resizing
        boxes = torch.as_tensor(boxes, dtype=torch.float32).reshape(-1, 4)
        boxes[:, 2:] += boxes[:, :2]
        boxes[:, 0::2].clamp_(min=0, max=w)
        boxes[:, 1::2].clamp_(min=0, max=h)

        classes = [obj["category_id"] for obj in anno if obj["segmentation"]]
        classes = torch.tensor(classes, dtype=torch.int64) - 1 # for labels between 0 and 9

        segmentations = [obj["segmentation"] for obj in anno if obj["segmentation"]]
        masks = self.convert_coco_poly_to_mask(segmentations=segmentations, height=h, width=w) # this requires investigation
                                                                                               # if sample 82 is not an isolated case
        keep = (boxes[:, 3] > boxes[:, 1]) & (boxes[:, 2] > boxes[:, 0])

        #print(keep)
        #print(keep.shape)
        #print(masks)

        boxes = boxes[keep]
        classes = classes[keep]
        masks = masks#[keep] # some samples could not be loaded

        target = {}
        target["boxes"] = boxes
        target["labels"] = classes
        target["masks"] = masks
        target["image_id"] = image_id

        # for conversion to coco api
        area = torch.tensor([obj["area"] for obj in anno if obj["segmentation"]])
        iscrowd = torch.tensor(
            [obj["iscrowd"] if "iscrowd" in obj else 0 for obj in anno if obj["segmentation"]]
        )
        target["area"] = area[keep]
        target["iscrowd"] = iscrowd[keep]

        target["orig_size"] = torch.as_tensor([int(h), int(w)])
        target["size"] = torch.as_tensor([int(h), int(w)])

        return image, target

    def convert_coco_poly_to_mask(self, segmentations, height, width):
      masks = []
      for polygons in segmentations:
          rles = coco_mask.frPyObjects(polygons, height, width)

          mask = coco_mask.decode(rles)
          if mask.max() < 1:
            continue
          if len(mask.shape) < 3:
            mask = mask[..., None]
          mask = torch.as_tensor(mask, dtype=torch.uint8)
          mask = mask.any(dim=2)
          masks.append(mask)
      if masks:
          masks = torch.stack(masks, dim=0)
      else:
          masks = torch.zeros((0, height, width), dtype=torch.uint8)
      return masks


def make_coco_transforms(image_set):

    normalize = T.Compose(
        [T.ToTensor(), T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]
    )
    #max_size = 1333
    max_size = 1024
    scales = [480, 512, 544, 576, 608, 640, 672, 704, 736, 768, 800]
              # 832, 864, 896, 928, 960]

    if image_set == "train":
        return T.Compose(
            [
                T.RandomHorizontalFlip(),
                T.RandomSelect(
                    T.RandomResize(scales, max_size=max_size),
                    T.Compose(
                        [
                            # T.RandomResize([400, 500, 600])
                            T.RandomResize([2000, 2500, 3000]),
                            #T.RandomSizeCrop(384, 600),
                            T.RandomSizeCrop(1920, 3000),
                            T.RandomResize(scales, max_size=max_size),
                        ]
                    ),
                ),
                normalize,
            ]
        )



    if image_set == "val":
        return T.Compose([T.RandomResize([800], max_size=max_size), normalize,])
    if image_set == "test":
        return None

    raise ValueError(f"unknown {image_set}")

def create_dataset(split):
  ann_file = Path("/content/TACO-expl/data/annotations_off_0_" + split +".json")
  img_folder = Path("/content/MyDrive/MyDrive/official/")

  dataset = TACODataset(
      img_folder,
      ann_file,
      classes=classes,
      transforms=make_coco_transforms(split),
      local_rank=mutils.get_local_rank(),
      local_size=mutils.get_local_size(),
  )
  return dataset

In [12]:
train_dataset = create_dataset('train')
val_dataset = create_dataset('val')

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [13]:
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog

register_coco_instances("TACO_train", {}, train_dataset.annotation_file, train_dataset.image_dir)
MetadataCatalog.get("TACO_train").set(thing_classes=train_dataset.classes)
dataset_dicts_train = DatasetCatalog.get("TACO_train")

register_coco_instances("TACO_val", {}, val_dataset.annotation_file, val_dataset.image_dir)
MetadataCatalog.get("TACO_val").set(thing_classes=val_dataset.classes)
dataset_dicts_val = DatasetCatalog.get("TACO_val")

[05/02 09:48:14 d2.data.datasets.coco]: Loaded 1200 images in COCO format from /content/TACO-expl/data/annotations_off_0_train.json
[05/02 09:48:14 d2.data.datasets.coco]: Loaded 150 images in COCO format from /content/TACO-expl/data/annotations_off_0_val.json


In [ ]:
train_cfg = cfg_alt_dino.clone()
with open("/content/TACO-expl/MaskDINO/configs/taco/taco_train.yaml", "w") as f:
  f.write(train_cfg.dump())

In [16]:
train_cfg_loaded = get_cfg()
train_cfg_loaded.set_new_allowed(True)
train_cfg_loaded.merge_from_file("/content/TACO-expl/MaskDINO/configs/taco/taco_train.yaml")
print(train_cfg_loaded.SOLVER.IMS_PER_BATCH)

8


In [15]:
%cd /content/TACO-expl/MaskDINO/
from train_net import Trainer
!python -m pip install numpy==1.23.1
import numpy as np
np.bool = np.bool_

/content/TACO-expl/MaskDINO
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 49.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.86 requires numpy>=1.24.1, but you have numpy 1.23.1 which is incompatible.
pandas-stubs 2.0.3.230814 requires numpy>=1.25.0; python_version >= "3.9", but you have numpy 1.23.1 which is incompatible.
tensorflow 2.15.0 requires numpy<2.0.0,>=1.23.5, but you have numpy 1.23.1 which is incompatible.


In [ ]:
trainer = Trainer(train_cfg_loaded)
trainer.resume_or_load()  # load last checkpoint or MODEL.WEIGHTS
trainer.train()

criterion.weight_dict  {'loss_ce': 4.0, 'loss_mask': 5.0, 'loss_dice': 5.0, 'loss_bbox': 5.0, 'loss_giou': 2.0, 'loss_ce_interm': 4.0, 'loss_mask_interm': 5.0, 'loss_dice_interm': 5.0, 'loss_bbox_interm': 5.0, 'loss_giou_interm': 2.0, 'loss_ce_dn': 4.0, 'loss_mask_dn': 5.0, 'loss_dice_dn': 5.0, 'loss_bbox_dn': 5.0, 'loss_giou_dn': 2.0, 'loss_ce_interm_dn': 4.0, 'loss_mask_interm_dn': 5.0, 'loss_dice_interm_dn': 5.0, 'loss_bbox_interm_dn': 5.0, 'loss_giou_interm_dn': 2.0, 'loss_ce_0': 4.0, 'loss_mask_0': 5.0, 'loss_dice_0': 5.0, 'loss_bbox_0': 5.0, 'loss_giou_0': 2.0, 'loss_ce_interm_0': 4.0, 'loss_mask_interm_0': 5.0, 'loss_dice_interm_0': 5.0, 'loss_bbox_interm_0': 5.0, 'loss_giou_interm_0': 2.0, 'loss_ce_dn_0': 4.0, 'loss_mask_dn_0': 5.0, 'loss_dice_dn_0': 5.0, 'loss_bbox_dn_0': 5.0, 'loss_giou_dn_0': 2.0, 'loss_ce_interm_dn_0': 4.0, 'loss_mask_interm_dn_0': 5.0, 'loss_dice_interm_dn_0': 5.0, 'loss_bbox_interm_dn_0': 5.0, 'loss_giou_interm_dn_0': 2.0, 'loss_ce_1': 4.0, 'loss_mask_1':

maskdino_r50_50ep_300q_hid1024_3sd1_instance_maskenhanced_mask46.1ap_box51.5ap.pth: 527MB [00:02, 238MB/s]                           


[05/02 09:50:11 d2.checkpoint.c2_model_loading]: Following weights matched with model:
| Names in Model                                                         | Names in Checkpoint                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learnin

[05/02 09:52:53 d2.utils.events]:  eta: 28 days, 22:48:41  iter: 19  total_loss: 134.1  loss_ce: 4.36  loss_mask: 0.1375  loss_dice: 1.688  loss_bbox: 0.3492  loss_giou: 1.293  loss_ce_dn: 1.671  loss_mask_dn: 0.06469  loss_dice_dn: 1.586  loss_bbox_dn: 0.09398  loss_giou_dn: 0.6601  loss_ce_0: 5.095  loss_mask_0: 0.1375  loss_dice_0: 1.89  loss_bbox_0: 0.3755  loss_giou_0: 1.382  loss_ce_dn_0: 0.7755  loss_mask_dn_0: 0.5931  loss_dice_dn_0: 4.293  loss_bbox_dn_0: 0.2338  loss_giou_dn_0: 0.8595  loss_ce_1: 4.915  loss_mask_1: 0.1066  loss_dice_1: 1.811  loss_bbox_1: 0.369  loss_giou_1: 1.341  loss_ce_dn_1: 1.517  loss_mask_dn_1: 0.07814  loss_dice_dn_1: 1.69  loss_bbox_dn_1: 0.133  loss_giou_dn_1: 0.691  loss_ce_2: 4.607  loss_mask_2: 0.1185  loss_dice_2: 1.721  loss_bbox_2: 0.3744  loss_giou_2: 1.227  loss_ce_dn_2: 1.724  loss_mask_dn_2: 0.06952  loss_dice_dn_2: 1.602  loss_bbox_dn_2: 0.1117  loss_giou_dn_2: 0.6657  loss_ce_3: 4.502  loss_mask_3: 0.1101  loss_dice_3: 1.736  loss_bbox_

/usr/local/lib/python3.10/dist-packages/shapely/constructive.py:180: RuntimeWarning: divide by zero encountered in buffer
  return lib.buffer(


[05/02 10:22:32 d2.utils.events]:  eta: 18 days, 2:45:48  iter: 379  total_loss: 62.67  loss_ce: 1.423  loss_mask: 0.04746  loss_dice: 1.025  loss_bbox: 0.147  loss_giou: 0.7086  loss_ce_dn: 0.1779  loss_mask_dn: 0.04429  loss_dice_dn: 0.9614  loss_bbox_dn: 0.05711  loss_giou_dn: 0.4382  loss_ce_0: 1.624  loss_mask_0: 0.0562  loss_dice_0: 1.088  loss_bbox_0: 0.2205  loss_giou_0: 1.113  loss_ce_dn_0: 0.5377  loss_mask_dn_0: 0.3724  loss_dice_dn_0: 3.138  loss_bbox_dn_0: 0.2507  loss_giou_dn_0: 0.8576  loss_ce_1: 1.582  loss_mask_1: 0.05628  loss_dice_1: 1.044  loss_bbox_1: 0.164  loss_giou_1: 0.8025  loss_ce_dn_1: 0.2192  loss_mask_dn_1: 0.05568  loss_dice_dn_1: 1.013  loss_bbox_dn_1: 0.07976  loss_giou_dn_1: 0.4897  loss_ce_2: 1.534  loss_mask_2: 0.04905  loss_dice_2: 1.019  loss_bbox_2: 0.1829  loss_giou_2: 0.7794  loss_ce_dn_2: 0.1906  loss_mask_dn_2: 0.04604  loss_dice_dn_2: 1.004  loss_bbox_dn_2: 0.07291  loss_giou_dn_2: 0.4609  loss_ce_3: 1.564  loss_mask_3: 0.05369  loss_dice_3: 